In [1]:
library(data.table)
# library(pracma) # only needed for moving-average plot
load('../data/cancer_type_pd_th25.rda') # probabilities of each mutation type
load('../data/gene_pdt.rda')
dftmp <- read.csv('~/jackgl/analyses/tcga_gene_lists/frequently_mutated_genes_TCGA_brain.csv')
geneList_brain <- as.vector(dftmp$Symbol) # " " " " " for brain cancer
rm(dftmp)

In [2]:
head(gene_pdt)
geneList_brain

chrom,geneSym,p,type
1,DDX11L1,5.752612e-06,0
1,WASH7P,1.752652e-06,0
1,MIR6859-1,2.248431e-08,0
1,MIR1302-2,5.635693e-06,0
1,FAM138A,5.075834e-06,0
1,OR4G4P,1.019439e-05,0


[1] "IDH1"   "TP53"   "ATRX"   "TTN"    "PTEN"   "EGFR"   "CIC"    "MUC16" 
 [9] "FLG"    "PIK3CA"

In [3]:
gene_pdt[gene_pdt$geneSym %in% geneList_brain,]$type <- 1
head(gene_pdt[gene_pdt$geneSym %in% geneList_brain,])

chrom,geneSym,p,type
1,FLG,3.770620e-05,1
2,TTN,8.259759e-04,1
2,IDH1,4.563416e-05,1
3,PIK3CA,1.246800e-04,1
7,EGFR,8.076928e-05,1
10,PTEN,1.509484e-04,1


In [4]:
birthrate <- function(nld, nlp, npd, npp, sld, slp, spd, spp, tau) { # probability of birth per timestep   
    return(((((1+sld)^nld)*((1+spd)^npd))/(((1+slp)^nlp)*((1+spp)^npp)))*tau)
}
delta_ncells <- function(B, D, ncells) { # change in number of cells for a clone
    return (max(ncells + rbinom(1,ncells,min(B,1))-rbinom(1,ncells,min(D,1)),0))
}
get_mu_i <- function(B, mu) {return(mu*B)} # mutation rate of clone i: proportional to birth rate
get_nins <- function(ncells, mu_i) {return (rbinom(1,ncells,mu_i))} # number of insertions in current clone

In [5]:
update_mcount <- function(nld,nlp,npd,npp,typemut,typegene) {
    
    if (typemut==1) {return(list(nld,nlp,npd,npp))}
    else if (typemut==2 & typegene==0) {return(list(nld,nlp+1,npd,npp))}
    else if (typemut==2 & typegene==1) {return(list(nld+1,nlp,npd,npp))}
    else if (typemut==3 & typegene==0) {return(list(nld,nlp-1,npd,npp+1))}
    else if (typemut==3 & typegene==1) {return(list(nld-1,nlp,npd+1,npp))}
    
}

In [6]:
# This function will update the gene lists for a given clone

# type 1 - no effect
# type 2 - weak effect
# type 3 - strong effect

update_genes <- function(lgenes,pgenes) {
    lgenes<-unlist(lgenes)
    pgenes<-unlist(pgenes)
#     if (length(lgenes)==0) {return(list(list(lgenes),list(pgenes)))} # Shouldn't run into this?
    
    if (tail(lgenes,1) %in% pgenes) { # If the last gene in list is in list of paired disrupted genes, discard it
        lgenes <- head(lgenes,length(lgenes)-1)
        if (length(lgenes)==0) {lgenes <- c(NULL)}
        type<-1
    } else {
        if (tail(lgenes,1) %in% head(lgenes,length(lgenes)-1)) { # If the last gene is in the list of lone disrupted genes
            if (sample(c(0,1),1)) { # With 50% probability, assume it's the other copy, add the gene to the paired list, and discard it (all instances) from lone list
                pgenes <- append(pgenes,tail(lgenes,1))
                lgenes <- lgenes[lgenes!=tail(lgenes,1)]
                if (length(lgenes)==0) {lgenes <- c(NULL)}
                type<-3
            } else { # Else, assume it hit already disrupted copy, and discard from list
                lgenes <- head(lgenes,length(lgenes)-1)
                if (length(lgenes)==0) {lgenes <- c(NULL)}
                type<-1
            }
        } else {
            type<-2
        }
    }
    if (length(pgenes)==0) {pgenes <- list(NULL)}
    return(list(list(lgenes),list(pgenes),type))
}

In [7]:
run_sim <- function(N0, mu, tau, NT, sld, slp, spd, spp, gene_pdt, nclones, logpath) {

    # Allocate population
    Pop <- data.table(ncells=rep(0,nclones),
                      B=rep(0,nclones),
                      mu_i=rep(0,nclones),
                      nld=rep(0,nclones),
                      nlp=rep(0,nclones),
                      npd=rep(0,nclones),
                      npp=rep(0,nclones),
                      lgenes=rep(list(),nclones),
                      pgenes=rep(list(),nclones),
                      lasttype=rep(0,nclones))
    # Initialize population
    Pop[1:2,c('ncells','nld','nlp','npd','npp','lgenes','pgenes','lasttype'):=list(c(N0-1,1),
                                                                                 c(0,1),
                                                                                 c(0,0),
                                                                                 c(0,0),
                                                                                 c(0,0),
                                                                                 list(list(),list('default driver')),
                                                                                 list(list(),list()),
                                                                                 c(0,1))]
    
    # Assign birth and insertion rates
    Pop[1:2, B := mapply(birthrate, nld, nlp, npd, npp, sld, slp, spd, spp, tau)]
    Pop[1:2, mu_i := mapply(get_mu_i, B, mu)]
    
    # For if the population data table needs to be enlarged
    bkup_Pop <- data.table(ncells=rep(0,nclones),
                      B=rep(0,nclones),
                      mu_i=rep(0,nclones),
                      nld=rep(0,nclones),
                      nlp=rep(0,nclones),
                      npd=rep(0,nclones),
                      npp=rep(0,nclones),
                      lgenes=rep(list(),nclones),
                      pgenes=rep(list(),nclones),
                      lasttype=rep(0,nclones))
    
    N <- rep(0,NT) # Allocate array for population size time series
    genes <- rep(NA,nrow(gene_pdt))
    write('Initialized...',file=logpath,append=TRUE)

    ptm <- proc.time()
    for (ii in 1:NT) { # Loop over time steps
        
        if(ii %in% c(round(NT/4),round(NT/4*2),round(NT/4*3),NT)) { # Print progress at 25% completed intervals
            write(paste0(toString(ii/NT*100),'% done | ',format((proc.time()-ptm)[1],nsmall=3),' (s)'),file=logpath,append=TRUE)            
        }
        clog <- Pop$ncells>0 # Get logical array for indices of active (# cells >0) clones
        
        nins <- sum(mapply(get_nins,Pop$ncells[clog],Pop$mu_i[clog])) # Get number of exonic insertions
        if (nins > 0) {
            
            rownew <- which(Pop$ncells==0)[1] # Find first row of the data table with ncells==0
            
            gene_ids <- sample(1:nrow(gene_pdt),nins,replace=TRUE,prob=gene_pdt$p)
            gene_list <- gene_pdt$geneSym[gene_ids]
            genes<-append(genes,gene_list)
#             print(gene_list)
            genetypes <- gene_pdt$type[gene_ids]

            clonesWIns <- sample(rep(1:nclones,Pop$ncells),nins,replace=FALSE) # List clone id (row number) of each cell; sample without replacement
            ctab <- table(clonesWIns)
            cids <- as.integer(names(ctab)) # Get row ids of sampled clones
            set(Pop,cids,1L,Pop[cids,1L] - as.integer(ctab)) # Remove cells from sampled clones

            # Populate the new rows representing new clones
            if(rownew+nins-1>nrow(Pop)){ # Enlarge the population data object if needed
                Pop<-rbind(Pop,bkup_Pop)
                write('Increased size of pop. object',file=logpath,append=TRUE)
            }
            new_inds <- rownew:(rownew+nins-1) # Row indices of new rows            
            
            Pop[new_inds, c("ncells","nld","nlp","npd","npp","lgenes","pgenes","lasttype"):=list(1, 
                                                                                                Pop[clonesWIns]$nld, 
                                                                                                Pop[clonesWIns]$nlp, 
                                                                                                Pop[clonesWIns]$npd, 
                                                                                                Pop[clonesWIns]$npp, 
                                                                                                mapply(append,Pop[clonesWIns]$lgenes,gene_list,SIMPLIFY=FALSE),
                                                                                                Pop[clonesWIns]$pgenes,
                                                                                                genetypes)]
            # Update gene lists
            tmp1 <- t(mapply(update_genes,Pop[new_inds,]$lgenes,Pop[new_inds,]$pgenes))
            Pop[new_inds,]$lgenes <- tmp1[,1]
            Pop[new_inds,]$pgenes <- tmp1[,2]
            # Update insertion counts
            tmp2<-t(mapply(update_mcount,Pop[new_inds,]$nld,Pop[new_inds,]$nlp,Pop[new_inds,]$npd,Pop[new_inds,]$npp,tmp1[,3],Pop[new_inds]$lasttype))
            Pop[new_inds,c("nld","nlp","npd","npp"):=list(unlist(tmp2[,1]),unlist(tmp2[,2]),unlist(tmp2[,3]),unlist(tmp2[,4]))]
            # Update birth and insertion rates
            Pop[new_inds, B := mapply(birthrate, nld, nlp, npd, npp, sld, slp, spd, spp, tau)]
            Pop[new_inds, mu_i := mapply(get_mu_i, B, mu)]
        }

        N[ii] <- sum(Pop$ncells) # Get current number of cells
        if (N[ii]>=3*N0) {break} # Simulation stops if population has grown by 3X
        D <- N[ii]*tau/N0        # Compute death rate
        Pop[Pop$ncells>0, ncells:=mapply(delta_ncells, B, D, ncells)] # Update number of cells for all clones
        Pop <- Pop[order(Pop$ncells,decreasing=TRUE),] # Order data.table by ncells

    }
#     print(proc.time() - ptm)
    genes <- genes[!is.na(genes)]
    Pop <- Pop[,1:9]
    return(list(Pop,N,genes))

}


### Define parameters

In [8]:
# Initial number of cells
N0 <- 1000
# Probability of passenger or driver L1 insertion per cell cycle
mu <- 1
mu <- mu*(1-pd_lung[3]) # Scale by 1-probability of null insertion


spd <- .1
spp <- .001
sld <- spd*0.2
slp <- spp*0.2


# Number of time steps to simulate
NT <- 20000
# Time resolution: number of timesteps per cell cycle in the initial population (can also be interpreted as max possible fold change in cell cycle rate)
tau <- 4
tau <- 1/tau
# Buffer size of population data object; represents the max possible number of clones in the population
maxNClones <- max(2000,N0*5)

## Test run

In [ ]:
out <- run_sim(N0, mu, tau, NT, sld, slp, spd, spp, gene_pdt, maxNClones, './testwgene.log')
Pop <- out[[1]]
N <- out[[2]]

head(Pop)
N[N==0]<- NA
plot(1:NT,N,type='l',xlab='Timestep',ylab='Population Size')
# plot((1:NT)*tau,movavg(N,200,'s'),type='l',xlab='Generation',ylab='Population Size')

## Batch run

In [ ]:
# load('../data/cancer_type_pd_th25.rda') # probabilities of each mutation type
nrun <- 0
muv <- c(1e-1,1e-0,1e1,5e1)
muv_adj <- muv*(1-pd_lung[3])
N0v <- c(1e2,5e2,1e3,2e3)
outPath <- '../../test_brain_genetrack/run'
for (ii in 1:length(N0v)){
    for (jj in 1:length(muv)) {
        for (kk in 1:5) {
            nrun <- nrun+1
            line <- paste0('Run: ',toString(nrun),'\tN0: ',toString(N0v[ii]),'\tmu: ',toString(muv[jj]))
            write(line,file=paste0(outPath,'.log'),append=TRUE)
            out <- run_sim(N0v[ii], muv_adj[jj], tau, NT, sld, slp, spd, spp, gene_pdt, N0v[ii]*4, paste0(outPath,'.log'))
            Pop <- out[[1]]
            N <- out[[2]]
            genes <- out[[3]]
            save(Pop,N,genes,file=paste0(outPath,'_n0',N0v[ii],'_',nrun,".rda"))
            rm(Pop,N,genes)
        }
    }
}